In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
import torch
from dataset import PlanTDataset, generate_batch
from util.viz_batch import viz_batch

In [ ]:
import yaml
# Read YAML file
with open("config/config.yaml", 'r') as stream:
    cfg = yaml.safe_load(stream)

with open("config/model/PlanT.yaml", 'r') as stream:
    plnt = yaml.safe_load(stream)

plnt["training"]["input_bev"] = False
plnt["training"]["only_affects"] = True

plnt["training"]["augment"] = False
plnt["training"]["augment_parked"] = False

cfg["model"] = plnt

cfg["visualize"] = False

cfg["trainset_size"] = 1

In [ ]:
# plnt["training"]["augment"] = False
# plnt["training"]["augment_parked"] = False

In [ ]:
class DictAsMember(dict):
    def __getattr__(self, name):
        value = self[name]
        if isinstance(value, dict):
            value = DictAsMember(value)
        return value

cfg = DictAsMember(cfg)

In [ ]:
# from diskcache import Cache
# shared_dict = Cache(directory="/scratch_local/gwb301-1688311/tmp/ds_cache" ,size_limit=int(16 * 1024 ** 3))

In [ ]:
ds = PlanTDataset("/home/geiger/gwb301/code/PlanT_2_cleanup/results/data/garage_v2_2025_08_31/data", cfg)

In [ ]:
range_front = plnt["training"]["range"] * plnt["training"]["range_factor_front"]
range_sides = plnt["training"]["range"]

In [ ]:
routes = []
for i, x in enumerate(ds.BEV):
    if "0005" == x.decode()[-8:-4]:
        routes.append(i)

In [ ]:
routes.append(len(ds))

In [ ]:
counter = 0

In [ ]:
counter -= 1

In [ ]:
for i in range(routes[counter], routes[counter+1], 11): # np.random.randint(0, len(ds) , 10):
    print(i, ds.BEV[i])

    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(10, 7)

    data = ds[i]
    batch = generate_batch([data])#[ds[i] for i in range(i, i+256)])
    batch["pred_path"] = batch["route"].numpy()
    img_1 = cv2.cvtColor(viz_batch(batch, pix_per_m=5, range_front=range_front, range_sides=range_sides), cv2.COLOR_RGB2BGR)
    
    data["input"] = [[y[0]] + x + y[-2:] for x, y in zip(ds.unquantize_box(data["output"]), data["input"])] # Use extents from original
    # del data["BEV"]
    batch = generate_batch([data])#[ds[i] for i in range(i, i+256)])
    batch["pred_path"] = [[]]
    batch["waypoints"] = [[]]
    batch["route_original"] = torch.tensor([[]])
    img_2 = cv2.cvtColor(viz_batch(batch, pix_per_m=5, range_front=range_front, range_sides=range_sides), cv2.COLOR_RGB2BGR)

    ax1.imshow(img_1)
    ax2.imshow(cv2.addWeighted(img_1, 0.75, img_2, 1, 0))
    plt.show()
counter += 1

In [ ]:
data = ds[257]

In [ ]:
data["input"]

In [ ]:
data["output"]

In [ ]:
x

In [ ]:
all_routes = set()
for x in ds.BEV:
    all_routes.add("_".join(x.decode().split("/")[-3].split("_")[2:4]))

import os
import lxml.etree as ET
from collections import Counter

stats = Counter()
route_dir = "/home/geiger/gwb301/code/PlanT_2_cleanup/generated_routes"
for x in os.listdir(route_dir):
    folder = os.path.join(route_dir, x)
    if not os.path.isdir(folder):
        continue

    for route in os.listdir(folder + "/random_weather_seed_1"):
        if route[:-4] in all_routes:
            tree = ET.parse(os.path.join(folder, "random_weather_seed_1", route)).getroot()
            scenario_type = tree.find("route").find("scenarios").find("scenario").get("type")
            
            stats[(x, scenario_type)] += 1

scenario_stats = Counter()
for x in stats:
    scenario_stats[x[1]] += stats[x]